# **Step 2 - Challenge solution**

This is one of the possible solutions to the [Step 1's challenge](01_intro.ipynb).

The cell below is identical to the one you have already seen before so just run the cell and continue.

**Reminder:** In general, always remember to run each cell with SHIFT+ENTER or the little arrow (VS Code). 

In [4]:
import os
import datetime
from openai import AzureOpenAI
from dotenv import load_dotenv
from termcolor import cprint

# Environment setup
load_dotenv()
api_key=os.environ['AZURE_OPENAI_API_KEY']
api_version=os.environ['AZURE_OPENAI_API_VERSION']
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

# Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=api_key,  
  api_version = api_version
  )

# Invokes the LLM with provided system and user messages
def invoke_llm(system_message:str, user_message:str)->str:
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]
    response = client.chat.completions.create(
    model=deployment,    
    messages = messages,    
    temperature=0.3)
    return response.choices[0].message.content

Following are the modified prompts to handle the requests regarding the food service schedule.  

As you see, we simply added the information regarding the working schedule in the system prompt.  That's a simple solution.   

In [8]:
# Here is the predefined system prompt (change it as you wish! 🙂)
system_message = f'''
You are an assistant with expertise in food delivery information.
Your service is open every day from 9 AM to 9 PM except on Mondays.
Opening hours for special guests 8 AM to 10 PM.
Users 7560 to 7570 are special guests.
'''

# Here is the user request (just an example, try changing it! 🙂)
user_message="Can I make an order now to be delivered tomorrow at 9 PM?"

Then, since the LLM doesn't know what the current date and time are, we also provide that information as part of the input request.

## Try it 

Use the `invoke_llm()` function with the system message and user message: 

In [5]:
invoke_llm(system_message, user_message)

"Sure, I can help with that. Since our service is open every day from 9 AM to 9 PM except on Mondays, I can assist you with placing an order for tomorrow at 9 PM. However, please note that if tomorrow is a Monday, we won't be able to fulfill the order.\n\nCould you please confirm if tomorrow is a Monday? If not, I can proceed with the details of your order."

## Now try with the current time 

The LLM does not know the current time.  So what happens when you try to make an order "now" ? 

In [10]:
user_message="Can I make an order now? I'm user 7564."
invoke_llm(system_message, user_message)

'Yes, you can make an order now. As a special guest, our service is open for you from 8 AM to 10 PM. How can I assist you with your order?'

## How can you let the LLM know the current date & time ? 

The most obvious solution is to automatically provide the current date and time in the user prompt. 

In [11]:
# We add current date and time to the user request
now=datetime.datetime.now().strftime("%d/%m/%Y %H:%M")
input_request=f"Today is {now}. Respond to the following user request: {user_message}"

cprint(f"Input: {input_request}", "blue") 
response = invoke_llm(system_message, input_request)
cprint(response, "yellow")

Input: Today is 09/09/2024 08:53. Respond to the following user request: Can I make an order now? I'm user 7564.
Yes, you can make an order now. As a special guest, you have access to our services from 8 AM to 10 PM. How can I assist you with your order?


Not so complicated in the end no? 🙂

### **But...**

Can you identify some possible limitations of this approach?  



## **Another challenge for you**  😎

Let's assume you now wish to provide extended opening hours for VIP ('special') customers:
- If the user is among a list of *special* users (potentially thousands), the opening schedule is from 8 AM to 10 PM except Mondays.
- For all the other customers, the schedule remains unchanged.

*Examples* 

User 7564 is a special user, 0000 is not.
  
- *I am user 7564, what are today's working hours?* -> 8AM-10PM
- *what are today's working hours?* -> 9AM-9PM
- *what are today's working hours?, btw I am 7564...* -> 8AM-10PM
- *what are today's working hours?, I am user 0000* -> 9AM-9PM


How would you refactor your (or the above) solution to cover this additional requirement?

Take some time to find a solution by experimenting with the above cells. 
When done, go to [Notebook 3](03_get_schedule_tool.ipynb) where we will investigate a possible solution.